In [10]:
from acc2sf_offline import get_cadence
from media_player import Player
import numpy as np
import time
import pandas as pd
from collections import namedtuple
from os import getenv
from typing import List
from dotenv import load_dotenv
load_dotenv(dotenv_path='../.env')
#from HR_2_optimal_HR import *

Sample = namedtuple(typename='Sample', field_names=['hr', 'cadance'])

In [ ]:
# This module adapts music tempo to HR by the following rules:
# current HR & cadnece above/below optimal  -> speed up/down tempo
# every stabilization_period, a decision for changing the tempo will be taken
# this modules ignores warm up period
THREASHOLD_HR = getenv('THREASHOLD_HR') or 10 # Default value
THREASHOLD_CADANCE = getenv('THREASHOLD_CADANCE') or 10 # Default value

# initiate audio player
player = Player(['../recordings/running_track_sunshine_jaunt.mp3'])

# module initialization
stabilization_period = 10 # in sec. divisuble by 10
delta = 10
#warm_up_time = 100 # in sec
runner_age = 26
runner_max_HR = 220 - runner_age # TODO: find more accurate formula
low_HR, high_HR = 95.95, 134.33
optimalHR = np.mean([low_HR, high_HR])
 
# Start of Warming Up period
# time.sleep(warm_up_time)
# optimal_HR = HR_to_optimal_HR(low_HR, high_HR)
# End of Warming Up period

# averaging cadence and HR in spans of 15 sec
cadence = get_cadence("../recordings/gym_12kmh/ACC.csv", running=True) # assuming cadence is measured after warm up period
print(f'{cadence=}')
print(f'{np.count_nonzero(cadence)=}')
# cadence = cadence[:len(cadence)-len(cadence)%stabilization_period]
# cadence = cadence.reshape(-1, stabilization_period)
# cadence = np.mean(cadence, axis=1)

df = pd.read_csv("../recordings/gym_12kmh/HR.csv", skiprows=2, header=None, names=['hr']) #  the signal is in 1Hz
heart_rate_signal = df['hr'].to_numpy()
cadence = cadence[:len(heart_rate_signal)]
# df = np.concatenate(df['hr'].to_numpy())
# df = df[:len(df)-len(df)%stabilization_period]
# df = df.reshape(-1, stabilization_period)
# HR = np.mean(df, axis=1)

def mean_samples(samples: List[Sample]) -> Sample:
    avg_sample = Sample(0,0)
    num_samples = len(samples)
    for hr, cadance in samples:
        avg_sample = Sample(avg_sample.hr+hr, avg_sample.cadance+cadance)
    return Sample(avg_sample.hr/num_samples, avg_sample.cadance/num_samples)


samples = [Sample(*s) for s in zip(heart_rate_signal, cadence)]
samples = [mean_samples(samples[i*stabilization_period:(i+1)*stabilization_period]) for i in range(len(samples//stabilization_period))]


for sample in samples:
    hr, cadance = sample
    hr_divergence = 100*(optimalHR - hr)/optimalHR
    cadence_divergence = 100*(player.current_song.tempo-cadence*60)/player.current_song.tempo # beats per minute
    if(hr_divergence > THREASHOLD_HR):
        if(cadence_divergence < THREASHOLD_CADANCE):
            pass
        print(f'{sample=}: tempo down')
    elif(hr < -THREASHOLD_HR):
        print(f'{sample=}: tempo up')
    else:
        print(f'{sample=}: tempo unchanged')
    # stabilization period
    # time.sleep(stabilization_period)